In [27]:
from copy import deepcopy
import os
import math
import shutil
import re
from glob import glob
from collections import defaultdict, Counter

import numpy as np
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_colwidth', None)

import rispy
mapping = deepcopy(rispy.TAG_KEY_MAPPING)
mapping['M2'] = 'extra'
mapping['AB'] = 'orig_abstract'
mapping['CN'] = 'call_number'
mapping['CH'] = 'chapters'
mapping['AS'] = 'articles'

In [28]:
dump_dir = '../data/bntl-db-csv-dump-4March2024'

### Recensies

*Add to reviews which publication they review (under "RI" = reviewed item).*

We first load the original data table for all publications in the original dump:

In [29]:
publications = pd.read_csv(f"{dump_dir}/publications.csv", header=0, parse_dates=['creation_date', 'modification_date'])
publications['id'] = publications['id'].fillna('')
publications.sample(3)

,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
13383,162814,841142351,tijdschriftofreeks,1979-,"Nehalennia: bulletin van de Werkgroep Historie en Archeologie van het Koninklijk Zeeuwsch Genootschap der Wetenschappen en de Zeeuwse Vereniging voor Dialectonderzoek. Afl. 31 (lente 1979) - .... Middelburg: Koninklijk Zeeuwsch Genootschap der Wetenschappen, Werkgroep Historie en Archeologie, 1979-.... .",ISSN 0168-7131.\r\nAfkorting: Nehalennia (Middelbg.)\r\nBewerkt: (1979) 31 - (2003) 142\r\n\r\nVerschijnt 4x per jaar.\r\nVoortzetting van: Bulletin van de werkgroep historie en archeologie onder auspiciën van het Koninklijk Zeeuwsch Genootschap der wetenschappen = ISSN 1384-7597.,NaN,NaN,NaN,NaN,1,NaT,19.0,2015-04-13 08:38:08,nehalennia_bulletin_van_de_wer
199764,108005,NaN,recensie,2000,"Fleurkens, A. C. G.\r\nIn: De zeventiende eeuw (Hilversum): 16 (2000) 2, 168-169.",NaN,NaN,NaN,NaN,NaN,1,NaT,18.0,2013-05-14 08:31:49,fleurkens_a_c_g_in_de_zeventie/fleurkens_anna_catharina_gerarda
220573,288464,NaN,recensie,2014,"Ewing, Dan.\r\nIn: Bijdragen en mededelingen betreffende de geschiedenis der Nederlanden/Low Countries historical review; BMGN-LCHR: 129 (2014) 2.",Online verschenen recensie.,NaN,NaN,NaN,NaN,17,2014-12-17 11:44:01,17.0,2014-12-17 11:45:56,ewing_dan_in_bijdragen_en_mede/ewing_dan


We are going to extract title descriptions for including that in the RIS dump, so we much sure that these are clean:

In [30]:
publications['title_description'] = [' '.join(p.split()).strip() for p in publications['title_description']]

We extract a review's title description and check to which (source) id this review was mapped:

In [31]:
#publications[publications['title_description'] == 'Kestemont, Mike. Eind goed, al goed? In: Queeste: 18 (2011) 1, 81-84.']['id']
publications[publications['title_description'] == 'Een waaier van emoties. Nijmegen: Vantilt. 23 p. Speciaal nummer van: Filter (Nijmegen): 25 (2018) 4 (dec) 3-26.']


,id,ppn,type,jaar,title_description,abstract,url,containing_publication_id,containing_publication_ppn,page_numbers,creator,creation_date,modifier,modification_date,canonical_url
2973,314016,NaN,specialetijdschriftaflevering,2018,Een waaier van emoties. Nijmegen: Vantilt. 23 p. Speciaal nummer van: Filter (Nijmegen): 25 (2018) 4 (dec) 3-26.,"P. 3 Inleiding, door de redactie; p. 26 Noten; p. 26 Bibliografie ""Koudwatervrees"".",NaN,NaN,NaN,NaN,451,2023-12-06 09:35:47,451.0,2023-12-06 10:03:06,een_waaier_van_emoties_nijmege/n_v_t


Next, we look up the (target) ID of the publication that was reviewed:

In [32]:
print(publications[publications['title_description'] == "Draux, Roland. Beatrijs' biecht: stilistisch en semiotisch onderzoek. Köln: Lambert Academic Publishing, 2009. 253 p."]['id'])

181127    273488
Name: id, dtype: int64


We now create three lookup dicts:
- `title2id`: to map a title description to a publication ID;
- `title2ptype`: to map a publication's title descirption to its publication type;
- `id2title`: to map a publication ID to a title description.

In [33]:
title2id = dict(zip(publications['title_description'], publications['id']))
id2title = dict(zip(publications['id'], publications['title_description']))

In [34]:
title2ptypes = defaultdict(set)
for td, pt in zip(publications['title_description'], publications['type']):
    title2ptypes[td].add(pt)

We now create a linkage dict (`review2reference`), that maps the IDs of reviews, to the IDs of the publication that they review:

In [35]:
references = pd.read_csv(f'{dump_dir}/publication_publications.csv')
reference2publication = dict(zip(references['publication_id'], references['referenced_publication_id']))

Now, we inverse our lookup dictionary, to that books can be mapped to a list of their reviews:

In [36]:
publication2references = defaultdict(list)
for review_id, book_id in reference2publication.items():
    publication2references[book_id].append(review_id)

def get_reviews(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    reviews = []
    for potential_review_id in publication2references[pub_id]:
        potential_review_title = id2title[potential_review_id]
        if 'recensie' in title2ptypes[potential_review_title]:
            try:
                reviews.append(potential_review_title)
            except KeyError:
                continue
    
    if reviews:
        return ' | '.join(reviews)
    

def get_chapters(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    chapters = []
    for potential_chapter_id in publication2references[pub_id]:
        potential_chapter_title = id2title[potential_chapter_id]
        if 'artikelboek' in title2ptypes[potential_chapter_title]:
            try:
                chapters.append(potential_chapter_title)
            except KeyError:
                continue
    
    if chapters:
        return ' | '.join(chapters)
    
def get_articles(pub_title):
    try:
        pub_id = title2id[pub_title]
    except KeyError:
        return None
    
    articles = []
    for potential_article_id in publication2references[pub_id]:
        potential_article_title = id2title[potential_article_id]
        if 'artikeltijdschrift' in title2ptypes[potential_article_title]:
            try:
                articles.append(potential_article_title)
            except KeyError:
                continue
    
    if articles:
        return ' | '.join(articles)


In [37]:
review_id = title2id["Kestemont, Mike. Eind goed, al goed? In: Queeste: 18 (2011) 1, 81-84."]
print(review_id)
book_id = reference2publication[review_id]
print(book_id)
book_title = id2title[book_id]
print(book_title)

274189
273488
Draux, Roland. Beatrijs' biecht: stilistisch en semiotisch onderzoek. Köln: Lambert Academic Publishing, 2009. 253 p.


In [38]:
t = "Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012. 318 p. (Middeleeuwse studies en bronnen; 131)."
print(get_reviews(t))
print(get_chapters(t))

Besamusca, Bart. De (late) Middelnederlandse ridderepiek belicht. In: Queeste (Hilversum): 20 (2013) 1, 45-49. | Bouwmeester, Gerard. Nieuwe inzichten in de late ridderepiek. In: Spiegel der letteren (Leuven): 55 (2013) 4, 537-539.
Janssens, Jef. De Middelnederlandse ridderepiek in de veertiende eeuw: postmodern of voorzichtig vernieuwend? In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 37-52. (Middeleeuwse studies en bronnen; 131). | Meulen, Janet F. van der. Vrouwen van Avesnes: een nieuwe Alexander in de Lage Landen. In: Ene andre tale: tendensen in de Middelnederlandse late ridderepiek; onder redactie van An Faems en Marjolein Hogenbirk. Hilversum: Verloren, 2012, p. 55-81. (Middeleeuwse studies en bronnen; 131). | Reynders, Anne. "Ghi heren, ic houde in ware wort dat ghi van Alexandre gehort hebt": de Middelnederlandse vertalingen van de Oudfranse "Florimont" en "Voeux du paon

In [39]:
llm_path = '../data/llm-dump'

for decade_folder in sorted(glob(f'{llm_path}/*')):
    print(':::', decade_folder, ':::')
    decade = decade_folder.split('/')[-1]
    
    for ris_path in sorted(glob(f'{decade_folder}/*_consolidated.ris')):
        print(ris_path)
        with open(ris_path) as f:
            entries = rispy.load(f, encoding='utf-8', mapping=mapping)
        
        for idx, entry in tqdm(list(enumerate(entries))):
            if 'extra' in entry:
                pub_title = ' '.join(entry['extra'].split()).strip()
            else:
                # this means that we weren't able to structure the item via the LLM...
                pub_title = ' '.join(entry['title'].split()).strip()
            
            # add to reviews which book they review:
            if 'keywords' in entry and 'recensie' in entry['keywords']:
                ref_title = None
                try:
                    pub_id = title2id[pub_title]
                    ref_id = reference2publication[pub_id]
                    ref_title = id2title[ref_id]
                except KeyError:
                    pass

                if ref_title:
                    entry['reviewed_item'] = ref_title
                    entries[idx] = entry
            
            reviews = get_reviews(pub_title)
            if reviews:
                entry['call_number'] = reviews
                entries[idx] = entry
            
            if 'boek' in title2ptypes[pub_title]:
                chapters = get_chapters(pub_title)
                if chapters:
                    entry['chapters'] = chapters
                    entries[idx] = entry

            if 'specialetijdschriftaflevering' in title2ptypes[pub_title]:
                articles = get_articles(pub_title)
                if articles:
                    entry['articles'] = articles
                    entries[idx] = entry

            if entry['type_of_reference'] == 'CHAP':
                try:
                    pub_id = title2id[pub_title]
                    containing_title = None
                
                    containing_id = reference2publication[pub_id]
                    containing_title = id2title[containing_id]
                except KeyError:
                    continue

                if containing_title:
                    entry['custom3'] = containing_title
                    entries[idx] = entry

                if containing_title:
                    entry['custom3'] = containing_title
                    entries[idx] = entry

            if entry['type_of_reference'] == 'JOUR' and 'extra' in entry:
                if 'speciaal' in entry['extra'].lower():
                    if 'keywords' in entry:
                        entry['keywords'].append('Bijdrage speciaal tijdschriftnummer')
                    else:
                        entry['keywords'] = ['Bijdrage speciaal tijdschriftnummer']

        out_ris_path = ris_path.replace('.ris', f'_link_{decade}.ris')
        with open(out_ris_path, 'w') as bibliography_file:
            rispy.dump(entries, bibliography_file, mapping=mapping)

::: ../data/llm-dump/1940s :::
../data/llm-dump/1940s/BOOK_consolidated.ris


100%|██████████| 1442/1442 [00:00<00:00, 214520.34it/s]


../data/llm-dump/1940s/CHAP_consolidated.ris


100%|██████████| 1764/1764 [00:00<00:00, 297174.45it/s]


../data/llm-dump/1940s/JFULL_consolidated.ris


...

100%|██████████| 6051/6051 [00:00<00:00, 217553.00it/s]


../data/llm-dump/misc/JOUR_consolidated.ris


100%|██████████| 3483/3483 [00:00<00:00, 282918.14it/s]


../data/llm-dump/misc/WEB_consolidated.ris


100%|██████████| 32/32 [00:00<00:00, 92182.51it/s]
